This code comes: https://github.com/hanbingyan/FVIOT/tree/main

In [1]:
import torch
import numpy as np
import random

random.seed(12345)
np.random.seed(12345)
torch.manual_seed(12345)
# check gpu is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Config
MEM_SIZE = 3000
BATCH_SIZE = 80
DISCOUNT = 1.0
N_INSTANCE = 10

Utility functions for a more compact code and for the optimisation of the nerual networks.

In [2]:
from collections import namedtuple
import torch.nn as nn

# def sinkhorn_knopp(mu, nu, C, reg, niter):
#     K = np.exp(-C/C.max()/reg)
#     u = np.ones((len(mu), ))
#     for i in range(1, niter):
#         v = nu/np.dot(K.T, u)
#         u = mu/(np.dot(K, v))
#     Pi = np.diag(u) @ K @ np.diag(v)
#     return Pi


Transition = namedtuple('Transition', ('time', 'x', 'y', 'value'))

class Memory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def clear(self):
        self.memory.clear()
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        samples = random.sample(self.memory, batch_size)
        return samples

    def __len__(self):
        return len(self.memory)


def optimize_model(policy_net, memory, optimizer, Trunc_flag):
    if len(memory) < BATCH_SIZE:
        return

    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    values_batch = torch.stack(batch.value)
    x_batch = torch.stack(batch.x)
    y_batch = torch.stack(batch.y)
    time_batch = torch.stack(batch.time)

    left_values = policy_net(time_batch, x_batch, y_batch)

    # # Compute the expected Q values
    Loss_fn = nn.SmoothL1Loss()
    # Loss_fn = nn.MSELoss()
    loss = Loss_fn(left_values, values_batch)

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    if Trunc_flag:
        for param in policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
    optimizer.step()
    return loss

The neural network at hand.

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import nbimporter

h = 8
class DQN(nn.Module):
    def __init__(self, x_dim, y_dim, T):
        super(DQN, self).__init__()
        self.T = T
        self.linear1 = nn.Linear(x_dim+y_dim, h)
        # self.linear1.weight.data.fill_(10.0)
        # torch.nn.init.xavier_uniform_(self.linear1.weight)
        # torch.nn.init.zeros_(self.linear1.weight)
        # torch.nn.init.zeros_(self.linear1.bias)
        # self.bn = nn.BatchNorm1d(h)
        self.linear2 = nn.Linear(h, h)
        # torch.nn.init.xavier_uniform_(self.linear2.weight)
        # torch.nn.init.zeros_(self.linear2.bias)
        # torch.nn.init.zeros_(self.linear2.weight)

        # self.dropout = nn.Dropout(p=0.5)

        self.linear3 = nn.Linear(h, 1)

        self.linear5 = nn.Linear(2, 1)
        # torch.nn.init.zeros_(self.linear5.bias)
        # torch.nn.init.zeros_(self.linear5.weight)
        # torch.nn.init.xavier_uniform_(self.linear5.weight)
        self.linear6 = nn.Linear(2, 1)
        # torch.nn.init.zeros_(self.linear6.bias)

    def forward(self, time, x, y):
        state = torch.cat((x, y), dim=1)
        state = torch.relu(self.linear1(state))
        # state = self.bn(state)
        state = torch.relu(self.linear2(state))
        # state = self.dropout(state)
        state = torch.sigmoid(self.linear3(state))
        time_f2 = torch.cat((self.T - time, (self.T - time)**2), dim=1)
        time_f1 =  self.linear5(time_f2)
        time_f2 = self.linear6(time_f2)
        return state*time_f1 + time_f2

The code to compute the Adapted Wasserstein distance (AW_2) between two brownian path.

In [ ]:
import torch.optim as optim
from torch.distributions.multivariate_normal import MultivariateNormal
import ot
import time as Clock

start = Clock.time()

####### One-dimensional case #########
# with parameter constraint
Trunc_flag = True
# No. of gradient descent steps (G)
N_OPT = 20
# No. of sample paths (N)
smp_size = 2000
# Sample size for empirical OT (B)
in_sample_size = 50

time_horizon = 4
x_dim = 1
y_dim = 1
x_vol = 1.0
y_vol = 0.5
x_init = 1.0
y_init = 2.0


###### Multidimensional case #########
## no parameter constraint
# Trunc_flag = False
# time_horizon = 5
# x_dim = 5
# y_dim = 5
# x_vol = 1.1
# y_vol = 0.1
# x_init = 1.0
# y_init = 2.0
# N_OPT = 400
# smp_size = 4000
# in_sample_size = 300


final_result = np.zeros(N_INSTANCE)

for n_ins in range(N_INSTANCE):

    val_hist = np.zeros(time_horizon+1)
    loss_hist = np.zeros(time_horizon+1)

    memory = Memory(MEM_SIZE)
    policy_net = DQN(x_dim, y_dim, time_horizon).to(device)
    target_net = DQN(x_dim, y_dim, time_horizon).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()
    # optimizer = optim.SGD(policy_net.parameters(), lr=0.1, momentum=0.9)
    optimizer = optim.Adam(policy_net.parameters(), lr=1e-2) # weight_decay=1e-3)

    x_path_pool = torch.zeros(smp_size, time_horizon+1, x_dim, device=device)
    y_path_pool = torch.zeros(smp_size, time_horizon+1, y_dim, device=device)
    x_path_pool[:, 0, :] = x_init
    y_path_pool[:, 0, :] = y_init

    for smp_id in range(smp_size):
        # sample many paths in advance
        for t in range(1, time_horizon + 1):
            x_path_pool[smp_id, t, :] = x_path_pool[smp_id, t - 1, :] + x_vol * torch.randn(x_dim, device=device)
            y_path_pool[smp_id, t, :] = y_path_pool[smp_id, t - 1, :] + y_vol * torch.randn(y_dim, device=device)

    for time in range(time_horizon, -1, -1):

        for smp_id in range(smp_size):
            x_mvn = MultivariateNormal(loc=x_path_pool[smp_id, time, :], covariance_matrix=torch.eye(x_dim, device=device)*x_vol**2)
            y_mvn = MultivariateNormal(loc=y_path_pool[smp_id, time, :], covariance_matrix=torch.eye(y_dim, device=device)*y_vol**2)
            next_x = x_mvn.sample((in_sample_size,))
            next_y = y_mvn.sample((in_sample_size,))

            x_batch = torch.repeat_interleave(next_x, repeats=in_sample_size, dim=0)
            y_batch = torch.tile(next_y, (in_sample_size, 1))
            l2_mat = torch.sum((x_batch - y_batch)**2, dim=1)

            if time == time_horizon:
                expected_v = 0.0
            elif time == time_horizon-1:
                min_obj = l2_mat.reshape(in_sample_size, in_sample_size)
                expected_v = ot.emd2(np.ones(in_sample_size) / in_sample_size, np.ones(in_sample_size) / in_sample_size,
                                     min_obj.detach().cpu().numpy())
            else:
                val = target_net(torch.ones(x_batch.shape[0], 1, device=device)*(time+1.0), x_batch, y_batch).reshape(-1)
                min_obj = (l2_mat + DISCOUNT*val).reshape(in_sample_size, in_sample_size)
                expected_v = ot.emd2(np.ones(in_sample_size)/in_sample_size, np.ones(in_sample_size)/in_sample_size,
                                     min_obj.detach().cpu().numpy())

            memory.push(torch.tensor([time], dtype=torch.float32, device=device), x_path_pool[smp_id, time, :],
                        y_path_pool[smp_id, time, :], torch.tensor([expected_v], device=device))

        # Optimize at time t
        for opt_step in range(N_OPT):
            loss = optimize_model(policy_net, memory, optimizer, Trunc_flag)
            if Trunc_flag:
                with torch.no_grad():
                    for param in policy_net.parameters():
                        ## param.add_(torch.randn(param.size(), device=device)/50)
                        param.clamp_(-1.0, 1.0)
            if loss:
                loss_hist[time] += loss.detach().cpu().item()


        loss_hist[time] /= N_OPT

        # update target network
        target_net.load_state_dict(policy_net.state_dict())
        # test initial value
        val = target_net(torch.ones(1, 1, device=device)*0.0, x_path_pool[0, 0, :].reshape(1, x_dim),
                         y_path_pool[0, 0, :].reshape(1, y_dim)).reshape(-1)
        val_hist[time] = val

        # empty memory
        memory.clear()
        print('Time step', time, 'Loss', loss_hist[time])

        # print('Shift vector in the last layer:', target_net.linear3.bias.sum().item())


    # for name, param in target_net.named_parameters():
    #     if param.requires_grad:
    #         print(name, param.data)


    print('Instance', n_ins)
    # print('Time elapsed', end - start)
    print('Last values', val_hist[0])
    final_result[n_ins] = val_hist[0]

print('All final value:', final_result)
print('Final mean:', final_result.mean())
print('Final std:', final_result.std())
end = Clock.time()
print('Average time for one instance:', (end-start)/N_INSTANCE)
# plt.figure(figsize=(8, 6))
# plt.plot(val_hist)
# plt.xlabel('Steps', fontsize=16)
# plt.ylabel(r'$V_0$', fontsize=16)
# # plt.tick_params(axis = 'both', which = 'major', labelsize = 16)
# plt.legend(bbox_to_anchor=(1, 1), title='', fontsize=16, title_fontsize=16)
# plt.savefig('conti_val.pdf', format='pdf', dpi=1000, bbox_inches='tight', pad_inches=0.1)
# plt.show()


# plt.figure(figsize=(8, 6))
# plt.plot(loss_hist)
# plt.xlabel('Steps', fontsize=16)
# plt.ylabel('Loss', fontsize=16)
# plt.savefig('conti_loss.pdf', format='pdf', dpi=1000, bbox_inches='tight', pad_inches=0.1)
# plt.show()

Time step 4 Loss 0.0006122382645116886
Time step 3 Loss 4.415977704524994
Time step 2 Loss 2.4819213271141054
Time step 1 Loss 1.2711868703365325
Time step 0 Loss 0.2843217607587576
Instance 0
Last values 6.015169620513916
Time step 4 Loss 0.0018359368987148628
Time step 3 Loss 3.56169570684433
Time step 2 Loss 2.42690686583519
Time step 1 Loss 1.010173037648201
Time step 0 Loss 0.5052285067737102
Instance 1
Last values 7.117129325866699
Time step 4 Loss 0.017255221891537074
Time step 3 Loss 3.9723653316497805
Time step 2 Loss 2.767244482040405
Time step 1 Loss 1.8418561697006226
Time step 0 Loss 1.3509894859045743
Instance 2
Last values 5.763694763183594
Time step 4 Loss 0.01835930245288182
Time step 3 Loss 3.9306265115737915
Time step 2 Loss 2.5592572927474975
Time step 1 Loss 0.7846329852938652
Time step 0 Loss 0.269201698154211
Instance 3
Last values 6.723105430603027
Time step 4 Loss 0.27581188827753067
